In [1]:
import sys
!{sys.executable} -m pip install sentinelsat

In [2]:
import os
import datetime
import json
import requests
import numpy as np
import matplotlib.pyplot as plt

import rasterio

from sentinelsat import SentinelAPI, geojson_to_wkt
from azure.storage.blob import ContainerClient

In [3]:
username = "sachindras"; password = "Scalalang29"
sas_token = os.path.expanduser('~/tokens/sentinel1_ro_sas.txt')

In [4]:
api_url = 'https://scihub.copernicus.eu/dhus'
platform_name = 'Sentinel-1'

# Query parameters
start_time = datetime.datetime(2020,1,1,0,0,0)
end_time = datetime.datetime(2021,2,1,0,0,0)

# URL containing a geojson file with an area of interest, in this case a rectangle around Seattle
query_geojson_url = 'https://ai4edatasetspublicassets.blob.core.windows.net/assets/stuff-near-seattle.geojson'

# Scaling constants that control rendering brightness
norm_value = 1500
composite_norm_value = 500

# When rendering whole images, how much should we downscale?
dsfactor = 10

In [5]:
if os.path.isfile(sas_token):
    lines = []
    with open(sas_token,'r') as f:
        lines = f.readlines()
    assert len(lines) >= 1
    sas_token = lines[0].strip()
    
storage_account_name = 'sentinel1euwest'
container_name = 's1-grd'

storage_account_url = 'https://' + storage_account_name + '.blob.core.windows.net/'

container_client = ContainerClient(account_url=storage_account_url, 
                                             container_name=container_name,
                                             credential=sas_token)

In [6]:
api = SentinelAPI(user=username, password=password, api_url=api_url)

In [7]:
geojson = json.loads(requests.get(query_geojson_url).text)
footprint = geojson_to_wkt(geojson)

products = api.query(footprint,
                     producttype='GRD',
                     date=(start_time,end_time),
                     platformname=platform_name)

Querying products:  51%|#####     | 100/197 [00:00<?, ?product/s]

In [8]:
product_values = list(products.values())
    
print('Found {} tiles:'.format(len(products)))
for p in product_values:
    print(p['filename'])

Found 197 tiles:
S1B_IW_GRDH_1SDV_20210129T020125_20210129T020150_025363_030553_F429.SAFE
S1B_IW_GRDH_1SDV_20210129T020150_20210129T020215_025363_030553_48CA.SAFE
S1B_IW_GRDH_1SDV_20210127T141257_20210127T141327_025341_0304A3_6E12.SAFE
S1B_IW_GRDH_1SDV_20210127T141232_20210127T141257_025341_0304A3_F6D6.SAFE
S1B_IW_GRDH_1SDV_20210124T015328_20210124T015353_025290_0302FD_BD63.SAFE
S1B_IW_GRDH_1SDV_20210120T142048_20210120T142113_025239_030157_02F1.SAFE
S1B_IW_GRDH_1SDV_20210117T020150_20210117T020215_025188_02FFBE_972E.SAFE
S1B_IW_GRDH_1SDV_20210117T020125_20210117T020150_025188_02FFBE_3068.SAFE
S1B_IW_GRDH_1SDV_20210115T141257_20210115T141327_025166_02FF0F_810D.SAFE
S1B_IW_GRDH_1SDV_20210115T141232_20210115T141257_025166_02FF0F_1170.SAFE
S1B_IW_GRDH_1SDV_20210112T015328_20210112T015353_025115_02FD69_43DB.SAFE
S1B_IW_GRDH_1SDV_20210108T142048_20210108T142113_025064_02FBBD_C037.SAFE
S1B_IW_GRDH_1SDV_20210105T020126_20210105T020151_025013_02FA23_7F71.SAFE
S1B_IW_GRDH_1SDV_20210105T020151_2

In [9]:
product = product_values[0]

In [10]:
import pprint; pp = pprint.PrettyPrinter(indent=2)
pp.pprint(product)

{ 'acquisitiontype': 'NOMINAL',
  'beginposition': datetime.datetime(2021, 1, 29, 2, 1, 25, 407000),
  'endposition': datetime.datetime(2021, 1, 29, 2, 1, 50, 407000),
  'filename': 'S1B_IW_GRDH_1SDV_20210129T020125_20210129T020150_025363_030553_F429.SAFE',
  'footprint': 'MULTIPOLYGON (((-123.969368 45.992237, -120.676697 46.3904, '
               '-121.011612 47.888786, -124.397346 47.490295, -123.969368 '
               '45.992237)))',
  'format': 'SAFE',
  'gmlfootprint': '<gml:Polygon '
                  'srsName="http://www.opengis.net/gml/srs/epsg.xml#4326" '
                  'xmlns:gml="http://www.opengis.net/gml">\n'
                  '   <gml:outerBoundaryIs>\n'
                  '      <gml:LinearRing>\n'
                  '         <gml:coordinates>47.490295,-124.397346 '
                  '47.888786,-121.011612 46.390400,-120.676697 '
                  '45.992237,-123.969368 '
                  '47.490295,-124.397346</gml:coordinates>\n'
                  '      </gml:Lin

In [11]:
root = 'GRD'

scene_name = product['filename'].replace('.SAFE','')
mode = scene_name[4:6]
polarization = scene_name[14:16] # "DV", for example, is "dual VV/VH"
year = scene_name[17:21];
month = scene_name[21:23].lstrip('0');
day = scene_name[23:25].lstrip('0');

azure_scene_prefix = '/'.join([root,year,month,day,mode,polarization,scene_name])

print('Azure scene prefix: {}'.format(azure_scene_prefix))

Azure scene prefix: GRD/2021/1/29/IW/DV/S1B_IW_GRDH_1SDV_20210129T020125_20210129T020150_025363_030553_F429


In [12]:
generator = container_client.list_blobs(name_starts_with=azure_scene_prefix)
scene_files = list(generator)
image_blobs = [blob.name for blob in scene_files if blob.name.endswith('.tiff')]
preview_blobs = [blob.name for blob in scene_files if blob.name.endswith('quick-look.png')]

print('Found {} image files and {} preview files:'.format(len(image_blobs),len(preview_blobs)))
for s in image_blobs:
    print(s)
for s in preview_blobs:
    print(s)    

AzureSigningError: Invalid base64-encoded string: number of data characters (37) cannot be 1 more than a multiple of 4